# Getting started - Requirements
Python (🐍):  This assumes that you have a python environment installed. If you haven't, we recommend installing [Anaconda](https://docs.anaconda.com/anaconda/install/).

FPsim (♀️): This tutorial uses [FPsim v2.0](https://github.com/fpsim/fpsim/tree/rc2.0-methodtime) (still under development). 

To install, you first need to clone or download the a copy of the source code from https://github.com/fpsim/fpsim
 
```
git clone -b rc2.0-methodtime --single-branch https://github.com/fpsim/fpsim.git
cd fpsim
pip install -e .
```

If it worked, you should be able to import fpsim with `import fpsim as fp`.

<div class="alert alert-info">
    
An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/fpsim/fpsim/blob/main/docs/tutorials/T4_intervention_eligibility.ipynb?install=fpsim) or [Binder](https://mybinder.org/v2/gh/fpsim/fpsim/HEAD?labpath=docs%2Ftutorials%2FT4_intervention_eligibility.ipynb).
    
</div>

## Interventions - how to select a specific subpopulation

At this point, you should now be able to run multiple simulations with distinct interventions, using default or custom parameters and plot the results. So far we've seen interventions that target a model parameter. Now we will see how we can we specify a function that tells FPsim who will receive the intervention. In particular, we will modify the 'state' value (e.g., education) of all eligible women. 


In [ ]:
import fpsim as fp
import sciris as sc
import matplotlib.pyplot as plt

In [ ]:
pars = dict(
    n_agents   = 1_000,
    location   = 'kenya',
    start_year = 2000, 
    end_year   = 2012,
    exposure_factor = 1.0  # Overall scale factor on probability of becoming pregnant
)


### Increase the number of women who have education

The `change_people_state` intervention can be used to change 'states' or attributes of women. Let's see how we could increase the number of women who achieve their education goals.

In [ ]:
def select_undereducated(sim):
    """ Select women who have a goal to achieve at least 1 year of education """
    is_eligible = ((sim.people.is_female) &
                   (sim.people.alive)     & 
                   (sim.people.edu_objective > 0))
    return is_eligible

In [ ]:
edu = fp.Education()
s0 = fp.Sim(pars=pars, education_module=edu, label='Baseline')

In [ ]:
change_education = fp.change_people_state(
                        'edu_attainment',
                        eligibility=select_undereducated,
                        years=2010.0,
                        new_val=15,  # Give all selected women 15 years of education
                    )
edu = fp.Education()
s1 = fp.Sim(pars=pars, 
            education_module=edu, 
            interventions=change_education,
            label='Increased education')

In [ ]:
msim = fp.parallel(s0, s1)
s0, s1 = msim.sims


In [ ]:
plt.plot(s0.results.t, s0.results.edu_attainment, label=s0.label)
plt.plot(s1.results.t, s1.results.edu_attainment, label=s1.label)
plt.ylabel('Average years of education among women')
plt.xlabel('Year')
plt.legend()